In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-mnsforest')

In [3]:
aoi = ee.FeatureCollection('projects/ee-mnsforest/assets/roi_mamminasata')

In [4]:
def mask_clouds(image):
    qa = image.select('QA_PIXEL')
    cloud = qa.bitwiseAnd(1 << 3).eq(0)
    shadow = qa.bitwiseAnd(1 << 4).eq(0)
    dilated = qa.bitwiseAnd(1 << 1).eq(0)
    cirrus = qa.bitwiseAnd(1 << 2).eq(0)
    mask = cloud.And(shadow).And(dilated).And(cirrus)
    return image.updateMask(mask)

In [5]:
def apply_scale_factors(image):
    optical = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(optical, None, True).addBands(thermal, None, True)

In [11]:
dataset = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('1994-01-01', '1994-12-31') \
    .map(mask_clouds) \
    .map(apply_scale_factors)

In [12]:
composite = dataset.median().clip(aoi)

In [13]:
ndvi = composite.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
evi = composite.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {'NIR': composite.select('SR_B4'), 'RED': composite.select('SR_B3'), 'BLUE': composite.select('SR_B1')}
).rename('EVI')
savi = composite.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
    {'NIR': composite.select('SR_B4'), 'RED': composite.select('SR_B3')}
).rename('SAVI')
ndwi = composite.normalizedDifference(['SR_B2', 'SR_B4']).rename('NDWI')
ndmi = composite.normalizedDifference(['SR_B4', 'SR_B5']).rename('NDMI')
ndbi = composite.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDBI')
bsi = composite.expression(
    '((SWIR + RED) - (NIR + BLUE)) / ((SWIR + RED) + (NIR + BLUE))',
    {
        'SWIR': composite.select('SR_B5'),
        'RED': composite.select('SR_B3'),
        'NIR': composite.select('SR_B4'),
        'BLUE': composite.select('SR_B1')
    }
).rename('BSI')
mndwi = composite.normalizedDifference(['SR_B2', 'SR_B5']).rename('MNDWI')

In [14]:
rgb_vis = {
    'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
    'min': 0.05,
    'max': 0.4
}

In [15]:
Map = geemap.Map()
Map.centerObject(aoi, 9)
Map.addLayer(composite, rgb_vis, 'Landsat 5 RGB')
Map

Map(center=[-5.211515044202248, 119.61581530927582], controls=(WidgetControl(options=['position', 'transparent…

In [16]:
sr_bands = composite.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).toFloat()

In [17]:
task1 = ee.batch.Export.image.toDrive(
    image=sr_bands,
    description='Export_L5_SR_Bands',
    folder='GEE Landsat 5',
    fileNamePrefix='L5_1994_SR_Bands',
    scale=30,
    region=aoi.geometry(),
    fileFormat='GeoTIFF'
)
task1.start()

In [18]:
def export_index(image, name):
    task = ee.batch.Export.image.toDrive(
        image=image.toFloat(),
        description=f'Export_{name}',
        folder='GEE Landsat 5',
        fileNamePrefix=f'L5_1994_{name}',
        scale=30,
        region=aoi.geometry(),
        fileFormat='GeoTIFF'
    )
    task.start()

In [19]:
export_index(ndvi, 'NDVI')
export_index(evi, 'EVI')
export_index(savi, 'SAVI')
export_index(ndwi, 'NDWI')
export_index(ndmi, 'NDMI')
export_index(ndbi, 'NDBI')
export_index(bsi, 'BSI')
export_index(mndwi, 'MNDWI')